# Creación de Consultas para los Endpoints de la API:

#### 1. Importar librerías y archivos necesarios:

In [89]:
import pandas as pd
import numpy as np

import warnings                    
warnings.filterwarnings("ignore") 

# Steam Games:
SGames = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')

# Users Items:
UItems = pd.read_csv('Clean_Data\\UItems_CD.csv', quotechar='"')

# Users Reviews:
UReviews = pd.read_csv('Clean_Data\\UReviews_CD.csv', quotechar='"')

# Users Reviews:
Genres_Items = pd.read_csv('Clean_Data\Genres_Items.csv', quotechar='"')

### 2. Crear la formula registros por desarrollador:

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

| Año  | Cantidad de artículos | Contenido Gratis |
|------|------------------------|-------------------|
| 2023 | 50                     | 27%               |
| 2022 | 45                     | 25%               |
| xxxx | xx                     | x%                |


2.1. Crear datos de entrada:

In [2]:
# Seleccionar las columnas deseadas y eliminar filas con datos nulos
End1_Developer = SGames[['id', 'developer', 'release_year', 'price']].dropna()
End1_Developer = End1_Developer.drop(End1_Developer[End1_Developer['release_year'] == 0].index) # así si
#End1_Developer = SGames[['id', 'developer', 'release_year', 'price']].replace(0, np.nan).dropna(subset=['release_year']) #ojo así no!



# Mostrar el DataFrame resultante
print("\nDataFrame final:")
End1_Developer



DataFrame final:


,id,developer,release_year,price
0,761140.0,Kotoshiro,2018,4.99
1,643980.0,Secret Level SRL,2018,0.00
2,670290.0,Poolians.com,2017,0.00
3,767400.0,彼岸领域,2017,0.99
5,772540.0,Trickjump Games Ltd,2018,3.99
...,...,...,...,...
32127,745400.0,Bidoniera Games,2018,1.99
32128,773640.0,"Nikita ""Ghost_RUS""",2018,1.99
32129,733530.0,Sacada,2018,4.99
32130,610660.0,Laush Dmitriy Sergeevich,2018,1.99


2.2. Obtener top-five de desarrolladores con más items para probar la fórmula:

In [3]:
# Calcular el top 5 de desarrolladores que más aparecen
top_developers = End1_Developer['developer'].value_counts().head(-5)

# Mostrar el top 5
print("Top 5 de desarrolladores que más aparecen:")
print(top_developers)

Top 5 de desarrolladores que más aparecen:
developer
Ubisoft - San Francisco                     1258
SmiteWorks USA, LLC                          808
Dovetail Games                               252
KOEI TECMO GAMES CO., LTD.                   213
Paradox Development Studio                   150
                                            ... 
Xform                                          1
Gustav                                         1
armogames                                      1
Patyuk Sergey                                  1
Facundo Balboa,Iván Piccione,Martín Díaz       1
Name: count, Length: 10568, dtype: int64


2.3. Crear fórmula "get_developer_stats":

In [4]:
def get_developer_stats(developer):
    
    # Filtrar el DataFrame por desarrollador
    developer_df = End1_Developer[End1_Developer['developer'] == developer]

    # Calcular la cantidad de items y el porcentaje de contenido gratuito por año
    stats_by_year = developer_df.groupby('release_year').agg({'id': 'count', 'price': lambda x: (x == 0).sum() / len(x) * 100}).reset_index()
   
    stats_by_year.columns = ['release_year', 'item_count', 'free_content_percentage']

    # Convertir los resultados a una lista de diccionarios
    result_list = stats_by_year.to_dict(orient='records')

    return result_list



2.4. Probar la fórmula:

In [5]:
# Ejemplo de uso de la función
print(get_developer_stats('Secret Level SRL'))

print(get_developer_stats('KOEI TECMO GAMES CO., LTD.'))

[{'release_year': 2018, 'item_count': 1, 'free_content_percentage': 100.0}]
[{'release_year': 2014, 'item_count': 28, 'free_content_percentage': 0.0}, {'release_year': 2015, 'item_count': 53, 'free_content_percentage': 0.0}, {'release_year': 2016, 'item_count': 5, 'free_content_percentage': 0.0}, {'release_year': 2017, 'item_count': 127, 'free_content_percentage': 7.086614173228346}]


2.5. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [6]:
# Exportar el DataFrame a CSV
End1_Developer.to_csv('Data_Queries/End1_Developer.csv', index=False)

print(f"DataFrame exportado a {'Data_Queries/End1_Developer.csv'}")

DataFrame exportado a Data_Queries/End1_Developer.csv


### 3. Crear función registros de usuario:

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.


Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

3.1. Crear datos de entrada:

In [56]:

# Crear el DataFrame End2_User
UItems_F= UItems[['user_id', 'item_id']]
End1_Developer_F=End1_Developer[['id','price']]
End2_User = pd.merge(UItems_F, End1_Developer, left_on='item_id', right_on='id')
End2_User = End2_User[['id', 'user_id', 'price']]
UReviews_F = UReviews[['item_id', 'user_id','recommend']]

# Agrupar End2_User en total de dienero gastado por usuario:
End2_User = End2_User.groupby('user_id')['price'].sum().reset_index()

# Mostrar el DataFrame resultante
print("\nDataFrame End2_User:")
print(End2_User.head(-5))

print("\nDataFrame UReviews_F:")
print(UReviews_F.head(-5))


DataFrame End2_User:
                 user_id    price
0                --000--   397.78
1                --ace--   166.82
2              --ionex--    99.93
3           -2SV-vuLB-Kg   427.50
4      -404PageNotFound-  1509.32
...                  ...      ...
68592     zyxwvutsrqponm   757.58
68593          zyzzchris    39.97
68594          zzbrunozz   239.86
68595              zzeee   501.64
68596            zzjang2   359.82

[68597 rows x 2 columns]

DataFrame UReviews_F:
       item_id            user_id  recommend
0         1250  76561197970982479       True
1        22200  76561197970982479       True
2        43110  76561197970982479       True
3       251610            js41637       True
4       227300            js41637       True
...        ...                ...        ...
49176   391460  76561198236893796      False
49177   209000  76561198236893796       True
49178      440           pigeonie       True
49179      730  76561198237650078       True
49180      730  7656119823

3.2. Crear fórmula "userdata":

In [71]:
def userdata(User_id):
    # Filtrar el DataFrame End2_User por user_id
    user_df = End2_User[End2_User['user_id'] == User_id]

    # Calcular la cantidad de dinero gastado por el usuario
    total_money_spent = user_df['price'].sum()

    # Calcular la cantidad de items
    total_items = len(user_df)

    # Filtrar el DataFrame UReviews_F por item_ids relacionados con el usuario
    user_reviews_df = UReviews_F[UReviews_F['user_id'] == User_id]

    # Calcular el porcentaje de recomendación basado en elementos únicos
    unique_items_count = user_reviews_df['item_id'].nunique()
    unique_items_with_recommendation = user_reviews_df[user_reviews_df['recommend'] == True]['item_id'].nunique()
    recommendation_percentage = (unique_items_with_recommendation / unique_items_count) * 100 if unique_items_count > 0 else 0

    # Crear el diccionario de resultados
    result_dict = {
        "Usuario": User_id,
        "Dinero gastado": f"{total_money_spent} USD",
        "% de recomendación": f"{recommendation_percentage:.2f}%",
        "Cantidad de items": total_items
    }

    return result_dict


3.3. Probar fórmula:

In [72]:
# Ejemplo de uso de la función
user_stats_result = userdata('76561197970982479')
print(user_stats_result)

{'Usuario': '76561197970982479', 'Dinero gastado': '3384.34 USD', '% de recomendación': '100.00%', 'Cantidad de items': 1}


3.4. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [73]:
# Exportar el DataFrame a CSV:

End2_User.to_parquet('Data_Queries/End2_User.parquet', index=False)
print(f"DataFrame exportado a {'Data_Queries/End2_User.parquet'}")

UReviews_F.to_csv('Data_Queries/UReviews_F.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/UReviews_F.csv'}")

DataFrame exportado a Data_Queries/End2_User.parquet
DataFrame exportado a Data_Queries/UReviews_F.csv


### 4. Función Usuario por Género de video Juego:

def UserForGenre( genero: str ) : Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas : 23}]}

4.1. Crear datos de Entrada:

In [42]:
# crear subset de SGames con las columnas necesarias:
subset_SGames = SGames[['id', 'release_year']]

# Unir subset_SGames y Genres_Items por la columna 'id':
merged_data = pd.merge(subset_SGames, Genres_Items, on='id')
merged_data

,id,release_year,genres
0,761140.0,2018,Action
1,761140.0,2018,Casual
2,761140.0,2018,Indie
3,761140.0,2018,Simulation
4,761140.0,2018,Strategy
...,...,...,...
80404,681550.0,0,Adventure
80405,681550.0,0,Indie
80406,681550.0,0,Action
80407,681550.0,0,Simulation


In [43]:
# Seleccionar las columnas 'user_id', 'item_id' y 'playtime_forever' de UItems_CD:
UItems_subset = UItems[['user_id', 'item_id', 'playtime_forever']]

# Unir UItems_CD_subset con merged_data por las columnas 'item_id' e 'id':
End3_Genre_User = pd.merge(UItems_subset, merged_data, left_on='item_id', right_on='id')

# Eliminar la columna duplicada 'id':
End3_Genre_User = End3_Genre_User.drop(columns=['id'])

# Eliminar filas con Año==0 en UserForGenre:
End3_Genre_User = End3_Genre_User[End3_Genre_User['release_year'] != 0]

# Mostrar el DataFrame resultante:
End3_Genre_User.head()


,user_id,item_id,playtime_forever,release_year,genres
0,76561197970982479,10,6,2000,Action
1,js41637,10,0,2000,Action
2,Riot-Punch,10,0,2000,Action
3,doctr,10,93,2000,Action
4,corrupted_soul,10,108,2000,Action


Se encuentra que los datos son demasiado grandes para guardarlos así que se procede a crear una dataframe agregado:

In [44]:
# Agrupa por género y usuario, y calcula la suma de playtime_forever
grouped_data = End3_Genre_User.groupby(['genres', 'user_id']).agg({'playtime_forever': 'sum'}).reset_index()

# Encuentra el usuario con la máxima suma de playtime_forever en cada género
max_playtime_users = grouped_data.groupby('genres')['playtime_forever'].idxmax()
max_playtime_data = grouped_data.loc[max_playtime_users]

# Ahora, agrupa por usuario y año y calcula la suma de playtime_forever
user_year_data = End3_Genre_User.groupby(['user_id', 'release_year']).agg({'playtime_forever': 'sum'}).reset_index()

# Fusiona los datos de max_playtime_data con user_year_data
End3_Genre_User = pd.merge(max_playtime_data, user_year_data, on='user_id', how='left')

# Muestra el DataFrame resultante
End3_Genre_User

,genres,user_id,playtime_forever_x,release_year,playtime_forever_y
0,Action,Sp3ctre,1692651,1993,0
1,Action,Sp3ctre,1692651,1995,651
2,Action,Sp3ctre,1692651,1996,0
3,Action,Sp3ctre,1692651,1997,283
4,Action,Sp3ctre,1692651,1998,0
...,...,...,...,...,...
259,others,Evilutional,691864,2013,142279
260,others,Evilutional,691864,2014,166485
261,others,Evilutional,691864,2015,33658
262,others,Evilutional,691864,2016,5164


4.2. Crear función UserForGenre:

In [45]:
def UserForGenre(genero: str):
    # Filtrar UserForGenre por el género dado
    genre_data = End3_Genre_User[End3_Genre_User['genres'] == genero]

    # Encontrar al usuario con más horas jugadas para el género dado
    max_hours_user = genre_data.loc[genre_data['playtime_forever_x'].idxmax()]['user_id']

    # Crear una lista de la acumulación de horas jugadas por año de lanzamiento
    hours_by_year = genre_data.groupby('release_year')['playtime_forever_y'].sum().reset_index()
    hours_by_year_list = [{"Año": int(year), "Horas": int(hours)} for year, hours in hours_by_year.values]

    # Crear el diccionario de resultados
    result_dict = {
        f"Usuario con más horas jugadas para el género {genero}": max_hours_user,
        "Horas jugadas": hours_by_year_list
    }

    return result_dict


4.3. Probar función:

In [46]:
# Ejemplo de uso
genero_ejemplo = "Action"  # Reemplazar con el género deseado
resultado = UserForGenre(genero_ejemplo)
print(resultado)

{'Usuario con más horas jugadas para el género Action': 'Sp3ctre', 'Horas jugadas': [{'Año': 1993, 'Horas': 0}, {'Año': 1995, 'Horas': 651}, {'Año': 1996, 'Horas': 0}, {'Año': 1997, 'Horas': 283}, {'Año': 1998, 'Horas': 0}, {'Año': 1999, 'Horas': 630}, {'Año': 2000, 'Horas': 70644}, {'Año': 2001, 'Horas': 250}, {'Año': 2002, 'Horas': 468}, {'Año': 2003, 'Horas': 7744}, {'Año': 2004, 'Horas': 128043}, {'Año': 2005, 'Horas': 21339}, {'Año': 2006, 'Horas': 182406}, {'Año': 2007, 'Horas': 115977}, {'Año': 2008, 'Horas': 8988}, {'Año': 2009, 'Horas': 114176}, {'Año': 2010, 'Horas': 129232}, {'Año': 2011, 'Horas': 693644}, {'Año': 2012, 'Horas': 671411}, {'Año': 2013, 'Horas': 475493}, {'Año': 2014, 'Horas': 324267}, {'Año': 2015, 'Horas': 1601021}, {'Año': 2016, 'Horas': 184414}, {'Año': 2017, 'Horas': 216356}]}


4.5. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [48]:
# Exportar el DataFrame a CSV:

End3_Genre_User.to_parquet('Data_Queries/End3_Genre_User.parquet', index=False)
print(f"DataFrame exportado a {'Data_Queries/End3_Genre_User.parquet'}")


DataFrame exportado a Data_Queries/End3_Genre_User.parquet


In [41]:
# Agrupa por género y usuario, y calcula la suma de playtime_forever
grouped_data = End3_Genre_User.groupby(['genres', 'user_id']).agg({'playtime_forever': 'sum'}).reset_index()

# Encuentra el usuario con la máxima suma de playtime_forever en cada género
max_playtime_users = grouped_data.groupby('genres')['playtime_forever'].idxmax()
max_playtime_data = grouped_data.loc[max_playtime_users]

# Ahora, agrupa por usuario y año y calcula la suma de playtime_forever
user_year_data = End3_Genre_User.groupby(['user_id', 'release_year']).agg({'playtime_forever': 'sum'}).reset_index()

# Fusiona los datos de max_playtime_data con user_year_data
End3_Genre_User = pd.merge(max_playtime_data, user_year_data, on='user_id', how='left')

# Muestra el DataFrame resultante
End3_Genre_User

,genres,user_id,playtime_forever_max_playtime,release_year,playtime_forever_user_year
0,Action,Sp3ctre,1692651,1993,0
1,Action,Sp3ctre,1692651,1995,651
2,Action,Sp3ctre,1692651,1996,0
3,Action,Sp3ctre,1692651,1997,283
4,Action,Sp3ctre,1692651,1998,0
...,...,...,...,...,...
259,others,Evilutional,691864,2013,142279
260,others,Evilutional,691864,2014,166485
261,others,Evilutional,691864,2015,33658
262,others,Evilutional,691864,2016,5164


### 5. Crear función de mejor desarrollador por año:

def best_developer_year( año: int ) : Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = Verdadero y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]


5.1. Crear los datos de entrada:

In [16]:
# Filtrar por recommend == True y sentiment_analysis == 2
filtered_reviews = UReviews[(UReviews['recommend'] == True) & (UReviews['sentiment_analysis'] == 2)]

# Resumir por el conteo de "item_id" por cada año "posted_year"
UReviews_gb = filtered_reviews.groupby(['posted_year', 'item_id']).size().reset_index(name='count')

# Mostrar el DataFrame resultante
UReviews_gb


,posted_year,item_id,count
0,2010,240,2
1,2010,300,1
2,2010,400,2
3,2010,440,9
4,2010,550,1
...,...,...,...
3842,2015,419070,1
3843,2015,421630,1
3844,2015,423120,1
3845,2015,423880,7


Solo se requiere saber el desarrollador y para ello se debe filtrar el dataframe de "SGames" y unir con la tabala anterior:

In [17]:
# Crear subconjunto de "SGames":
Games_subset= SGames[['developer','id']]

#Unir con la tabla "UReviews_gb"
End4_BestDeveloperYear= pd.merge(Games_subset, UReviews_gb, left_on='id', right_on='item_id')

# Seleccionar las columnas necesarias:
End4_BestDeveloperYear=End4_BestDeveloperYear[['developer', 'item_id','posted_year','count']]
End4_BestDeveloperYear

,developer,item_id,posted_year,count
0,Valve,70,2011,1
1,Valve,70,2012,2
2,Valve,70,2013,8
3,Valve,70,2014,16
4,Valve,70,2015,11
...,...,...,...,...
3359,"Epic Games, Inc.",13230,2012,1
3360,"Epic Games, Inc.",13230,2014,1
3361,"Epic Games, Inc.",13230,2015,2
3362,Valve,80,2013,1


5.2. Crear función "best_developer_year":

In [28]:
def best_developer_year(año: int):
    # Filtrar End4_BestDeveloperYear por el año dado
    filtered_data = End4_BestDeveloperYear[End4_BestDeveloperYear['posted_year'] == año]

    # Filtrar por juegos más recomendados (recommend = Verdadero y comentarios positivos)
    top_developers = filtered_data[filtered_data['count'] > 0]

    # Sumar los conteos por desarrollador y obtener el ranking de los 3 mejores desarrolladores
    top3_developers = top_developers.groupby('developer')['count'].sum().nlargest(3).reset_index()

    # Crear la lista de resultados en el formato deseado
    result_list = [{"Puesto {}".format(i + 1): developer} for i, developer in enumerate(top3_developers['developer'])]

    return result_list

5.3. Probar función:

In [29]:
# Ejemplo de uso
año_ejemplo = 2015  # Reemplazar con el año deseado
resultado = best_developer_year(año_ejemplo)
print(resultado)

{'Puesto 1': 'Valve', 'Puesto 2': 'Facepunch Studios', 'Puesto 3': 'Smartly Dressed Games'}


In [30]:
End4_BestDeveloperYear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3364 entries, 0 to 3363
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   developer    3292 non-null   object
 1   item_id      3364 non-null   int64 
 2   posted_year  3364 non-null   int64 
 3   count        3364 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 105.2+ KB


In [31]:
End4_BestDeveloperYear = End4_BestDeveloperYear.dropna(subset=['developer']) # Eliminar nulos por developer no tiene sentido dejarlos

5.4. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [75]:
# Exportar el DataFrame a CSV:

End4_BestDeveloperYear.to_csv('Data_Queries/End4_BestDeveloperYear.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/End4_BestDeveloperYear.csv'}")

DataFrame exportado a Data_Queries/End4_BestDeveloperYear.csv


### 6. Crear función que mida la percepción por desarrollador:

def desarrollador_reviews_analysis( desarrolladora: str ) : Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

6.1. Crear los datos de entrada:

In [21]:
# crear subconjunto de UReviews con el análisis de sentimiento:
sentimiento=UReviews[['item_id','sentiment_analysis']]

# Unimos data set creutilizando el subconjunto creado de Games_subset:
End5_Sentimiento=pd.merge(Games_subset, sentimiento, left_on='id', right_on='item_id')

# Seleccionamos columnas necesarias:
End5_Sentimiento=End5_Sentimiento[['developer','item_id','sentiment_analysis']]

# Revisamos dataframe:
End5_Sentimiento


,developer,item_id,sentiment_analysis
0,Valve,70,2
1,Valve,70,0
2,Valve,70,0
3,Valve,70,2
4,Valve,70,2
...,...,...,...
44734,Valve,80,1
44735,Valve,80,2
44736,Valve,80,2
44737,Valve,80,1


In [80]:
# Eliminar filas con valores nulos en cualquier columna
End5_Sentimiento = End5_Sentimiento.dropna()

In [85]:
# Reconstruir tabla para mejorar la lectura en los endpoints:

# Filtrar solo las reseñas positivas y negativas
positivas = End5_Sentimiento[End5_Sentimiento['sentiment_analysis'] == 2]
negativas = End5_Sentimiento[End5_Sentimiento['sentiment_analysis'] == 0]

# Contar el número de reseñas positivas y negativas por desarrollador
positivas_counts = positivas.groupby('developer').size().reset_index(name='Positive')
negativas_counts = negativas.groupby('developer').size().reset_index(name='Negative')

# Combinar los resultados en un nuevo DataFrame
resultado_final = pd.merge(positivas_counts, negativas_counts, on='developer', how='outer').fillna(0)

End5_Sentimiento = resultado_final

In [86]:
End5_Sentimiento

,developer,Positive,Negative
0,11 bit studios,20.0,20.0
1,14° East,1.0,1.0
2,17-BIT,2.0,0.0
3,1C Company,1.0,1.0
4,1C-Avalon,2.0,0.0
...,...,...,...
1590,bitHuffel,0.0,1.0
1591,momoStorm Entertainment,0.0,1.0
1592,nFusion Interactive,0.0,1.0
1593,souvenir circ.,0.0,1.0


6.2. Crear función "developer_reviews_analysis":

In [92]:
def desarrollador_reviews_analysis(ReviewsDeveloper: str):
   
    # Filtrar End5_Sentimiento por el desarrollador dado
    desarrollador_data = End5_Sentimiento[End5_Sentimiento['developer'] == ReviewsDeveloper]

    # Verificar si el desarrollador existe en el DataFrame
    if not desarrollador_data.empty:
        # Obtener la cantidad de reseñas positivas y negativas
        negative_count = int(desarrollador_data['Negative'])
        positive_count = int(desarrollador_data['Positive'])

        # Crear el diccionario de resultados
        result_dict = {ReviewsDeveloper: {'Negative': negative_count, 'Positive': positive_count}}

        return result_dict
    else:
        return {ReviewsDeveloper: {'Negative': 0, 'Positive': 0}}


6.3. Probar función:

In [93]:
# Ejemplo de uso
desarrolladora_ejemplo = 'Valve'  # Reemplazar con el nombre del desarrollador deseado
resultado = desarrollador_reviews_analysis(desarrolladora_ejemplo)
print(resultado)

{'Valve': {'Negative': 901, 'Positive': 5379}}


6.4. Guardar consulta (insumo para la API) en la carperta Data_Queries:

In [94]:
# Exportar el DataFrame a CSV:

End5_Sentimiento.to_csv('Data_Queries/End5_Sentimiento.csv', index=False)
print(f"DataFrame exportad  o a {'Data_Queries/End5_Sentimiento.csv'}")

DataFrame exportad  o a Data_Queries/End5_Sentimiento.csv
